In [16]:
from gurobipy import *

In [8]:
# Import necessary libraries
import json
import os

# Path to the input file generated by the Flask app
input_file = "inputs.json"

# Step 1: Load the input data
if os.path.exists(input_file):
    with open(input_file, "r") as f:
        data = json.load(f)
else:
    raise FileNotFoundError(f"{input_file} not found. Run the Flask app to save the inputs.")

# Extract values
warehousesNbre = data["warehousesNbre"]
customersNbre = data["customersNbre"]
fixed_costs = data["fixed_costs"]
transport_costs = data["transport_costs"]
variable_costs = data["variable_costs"]
penalty_costs = data.get("penalty_costs", [0] * customersNbre)  # Optional penalty costs
demands = data["demands"]
capacities = data["capacities"]
min_service = data.get("min_service", [1.0] * customersNbre)  # Default to 100% service level
fixed_costs = data["fixed_costs"]
transport_costs = data["transport_costs"]
# # Data
# warehousesNbre= int(input("Give the number of warehouses available: "))
# customersNbre= int(input("Give the number of customers (places): "))

warehouses = range(warehousesNbre)  # Index of warehouses
customers = range(customersNbre)  # Index of customers

# Fixed costs for opening each warehouse
#fixed_costs = list(map(int, input("Enter space-separated integers presenting fixed cost for each warehouse w: ").split()))
# Transportation costs: cost to serve each customer from each warehouse
#transport_costs = []
# for i in warehouses:
#     row = list(map(int, input(f"Enter space-separated values for row {i+1}: ").split()))
#     transport_costs.append(row)
# Step 4: Display the inputs
print("Number of Warehouses:", warehousesNbre)
print("Number of Customers:", customersNbre)
print("Fixed Costs:", fixed_costs)
print("Transportation Costs:")
for row in transport_costs:
    print(row)
print("Variable Costs:", variable_costs)
print("Penalty Costs:", penalty_costs)
print("Demands:", demands)
print("Capacities:", capacities)
print("Minimum Service Levels:", min_service)

# Customer demands (optional for extension, not used here directly)
#demands = [1, 1, 1, 1]  # Assume unit demand per customer

Number of Warehouses: 3
Number of Customers: 4
Fixed Costs: [1000, 1200, 1100]
Transportation Costs:
[10, 15, 20, 25]
[12, 18, 22, 28]
[14, 19, 21, 30]
Variable Costs: [5, 6, 7]
Penalty Costs: [50, 60, 70, 80]
Demands: [100, 200, 150, 300]
Capacities: [250, 250, 250]
Minimum Service Levels: [0.8, 0.9, 1.0, 0.7]


In [9]:
# Model
model = Model("Warehouse_Location")

# Decision variables
# Binary variables: 1 if warehouse w is open, 0 otherwise
open_warehouse = model.addVars(warehouses, vtype=GRB.BINARY, name="Open")

# Binary variables: 1 if customer c is served by warehouse w, 0 otherwise
serve_customer = model.addVars(warehouses, customers, vtype=GRB.BINARY, name="Serve")

Set parameter Username
Set parameter LicenseID to value 2594344


In [15]:
# Objective: Minimize total fixed and transportation costs
model.setObjective(
    quicksum(fixed_costs[w] * open_warehouse[w] for w in warehouses) +
    quicksum(transport_costs[w][c] * serve_customer[w, c] for w in warehouses for c in customers)+
    quicksum(variable_costs[r] * quicksum(serve_customer[r, d] for d in customers) for r in warehouses) +
    quicksum(penalty_costs[d] * (demands[d] - quicksum(serve_customer[r, d] for r in warehouses)) for d in customers),
    GRB.MINIMIZE
)
# Constraints

# 1. Each customer is served by exactly one warehouse
for c in customers:
    model.addConstr(quicksum(serve_customer[w, c] for w in warehouses) == 1, f"ServeCustomer_{c}")

# 2. A warehouse can serve a customer only if it is open
for w in warehouses:
    for c in customers:
        model.addConstr(serve_customer[w, c] <= open_warehouse[w], f"Link_{w}_{c}")

# 3. Non-negativity of served demand
model.addConstrs(
    serve_customer[r, d] >= 0 for r in warehouses for d in customers
)
# Optimize the model
model.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 44 rows, 15 columns and 84 nonzeros
Model fingerprint: 0x9e6da473
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 53446.000000
Presolve removed 28 rows and 0 columns
Presolve time: 0.01s
Presolved: 16 rows, 15 columns, 36 nonzeros
Variable types: 0 continuous, 15 integer (15 binary)
Found heuristic solution: objective 53430.000000

Root relaxation: objective 5.233000e+04, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incu

In [16]:
# Print results
if model.status == GRB.OPTIMAL:
    print("\nOptimal Solution Found:\n")
    for w in warehouses:
        if open_warehouse[w].x > 0.5:  # Open warehouses
            print(f"Warehouse in location {w+1} is open.")
    # for w in warehouses:
    #     for c in customers:
    #         if serve_customer[w, c].x > 0.5:  # Customers served by warehouses
    #             print(f"Customer {c+1} is served by Warehouse {w+1}.")
    print(f"\nTotal Cost = {model.objVal}")
else:
    print("No optimal solution found.")


Optimal Solution Found:

Warehouse in location 1 is open.

Total Cost = 52330.0
